In [ ]:
import os
import mlx_whisper
#from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from tqdm.notebook import tqdm
import time  # デモ用（実際の処理の進行が分かるようにするため）

# -------------------------
# 1. 音声ファイルから文字起こし
# -------------------------
print("📢 文字起こしを開始します...")

# tqdm を使用して進捗表示（デモ用）
for _ in tqdm(range(3), desc="音声処理中..."):
    time.sleep(1)  # 実際の処理時間の代わりにスリープを挿入

# 音声ファイルのパス
audio_data = 'test_recording.m4a'

# 専門用語・固有名詞のカスタムプロンプト
custom_prompt = (
    "以下の専門用語・固有名詞は必ず下記の表記で文字起こししてください。\n"
    "・『AI』は『エーアイ』\n"
    "・『Deep Learning』は『ディープラーニング』\n"
    "・『GPT』は『ジーピーティー』\n"
)

# 文字起こし実行
result = mlx_whisper.transcribe(
    audio_data,
    path_or_hf_repo="mlx-community/whisper-large-v3-mlx",
    prompt=custom_prompt
)

# **文の整形処理**
def format_text(text):
    import re
    text = re.sub(r"([。！？])", r"\1\n", text)  # 句読点で改行
    text = re.sub(r"、", "、 ", text)  # 読点の後にスペースを入れる
    return text.strip()
transcription_text = result['text']
# 文字起こし結果を保存
with open("transcription.txt", "w", encoding="utf-8") as f:
    f.write(transcription_text)

print("✅ 文字起こしが完了しました。")

# -------------------------
# 2. 文字起こし結果の要約
# -------------------------
print("📄 要約を作成中...")

# tqdm を使用して進捗表示
for _ in tqdm(range(3), desc="要約処理中..."):
    time.sleep(1)


# モデルのパスを指定（事前にダウンロードしたローカルパス）
model_path = "./models/bart-large-cnn"

# トークナイザーとモデルのロード
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# 要約用パイプラインを作成（local_files_only は削除）
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

# 要約実行
summary_result = summarizer(transcription_text, max_length=150, min_length=40, do_sample=False)
summary_text = summary_result[0]['summary_text']

# 要約結果を保存
with open("summary.txt", "w", encoding="utf-8") as f:
    f.write(summary_text)

print("✅ 要約が完了しました。")

# -------------------------
# 3. 英語翻訳（文字起こし結果・要約）
# -------------------------
print("🌍 英語翻訳を開始します...")

# tqdm を使用して進捗表示
for _ in tqdm(range(3), desc="翻訳処理中..."):
    time.sleep(1)

# 翻訳パイプライン
translator = pipeline(
    "translation",
    model="Helsinki-NLP/opus-mt-ja-en"
)

# 文字起こし結果の英語翻訳
english_transcription = translator(transcription_text, max_length=512)[0]['translation_text']
with open("transcription_en.txt", "w", encoding="utf-8") as f:
    f.write(english_transcription)

# 要約結果の英語翻訳
english_summary = translator(summary_text, max_length=512)[0]['translation_text']
with open("summary_en.txt", "w", encoding="utf-8") as f:
    f.write(english_summary)

print("✅ 英語翻訳が完了しました。")

# -------------------------
# 4. o3用プロンプト付き文書の作成
# -------------------------
print("📑 o3用のプロンプト付き文書を作成中...")

o3_prompt = (
    "以下は論文のアイデアの音声ファイルから文字起こしした内容です。\n"
    "内容について、o3の視点から意見や改善点を教えてください。\n\n"
)
o3_document = o3_prompt + transcription_text

# o3用文書を保存
with open("o3_prompt_document.txt", "w", encoding="utf-8") as f:
    f.write(o3_document)

print("✅ o3用の文書作成が完了しました。")

# -------------------------
# 完了メッセージ
# -------------------------
print("🎉 すべての処理が完了しました！")

In [ ]:
import numpy as np
print(np.__version__)

In [ ]:
!poetry add sentencepiece

In [11]:
import os
import time

import mlx_whisper
from tqdm.notebook import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

# 出力フォルダの作成
output_dir = "outputs"
os.makedirs(output_dir, exist_ok=True)
# -------------------------
# 1. 音声ファイルから文字起こし
# -------------------------
print("📢 文字起こしを開始します...")

for _ in tqdm(range(3), desc="音声処理中..."):
    time.sleep(1)

# 音声ファイルのパス
audio_data = "test_recording.m4a"


# 専門用語・固有名詞リストをファイルから読み込む
def load_custom_terms(file_path):
    terms = []
    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                if "," in line:
                    original, replacement = line.strip().split(",")
                    terms.append(f"・『{original.strip()}』は『{replacement.strip()}』")
    return "\n".join(terms)


# カスタムプロンプトを作成
terms_file = "custom_terms.txt"
custom_terms = load_custom_terms(terms_file)

custom_prompt = (
    "以下の専門用語・固有名詞は必ず下記の表記で文字起こししてください。\n"
    + custom_terms
)

# 文字起こし実行
result = mlx_whisper.transcribe(
    audio_data,
    path_or_hf_repo="mlx-community/whisper-large-v3-mlx",
    prompt=custom_prompt,
)


# **文の整形処理**
def format_text(text):
    import re

    text = re.sub(r"([。！？])", r"\1\n", text)  # 句読点で改行
    text = re.sub(r"、", "、 ", text)  # 読点の後にスペースを入れる
    return text.strip()


transcription_text = format_text(result["text"])

# 文字起こし結果を保存

# 文字起こし結果を保存
transcription_path = os.path.join(output_dir, "transcription.txt")
with open(transcription_path, "w", encoding="utf-8") as f:
    f.write(transcription_text)

print("✅ 文字起こしが完了しました。")

# -------------------------
# 2. 文字起こし結果の要約
# -------------------------
print("📄 要約を作成中...")

for _ in tqdm(range(3), desc="要約処理中..."):
    time.sleep(1)

summarizer_path = "./models/bart-large-cnn"

tokenizer = AutoTokenizer.from_pretrained(summarizer_path, local_files_only=True)
model = AutoModelForSeq2SeqLM.from_pretrained(summarizer_path, local_files_only=True)

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

summary_result = summarizer(
    transcription_text, max_length=150, min_length=40, do_sample=False
)
summary_text = format_text(summary_result[0]["summary_text"])

summary_path = os.path.join(output_dir, "summary.txt")
with open(summary_path, "w", encoding="utf-8") as f:
    f.write(summary_text)

print("✅ 要約が完了しました。")

# -------------------------
# 3. 英語翻訳（文字起こし結果・要約）
# -------------------------
# -------------------------
# 3. 英語翻訳（文字起こし結果・要約）
# -------------------------
print("🌍 英語翻訳を開始します...")

# tqdm を使用して進捗表示
for _ in tqdm(range(3), desc="翻訳処理中..."):
    time.sleep(1)

# 翻訳パイプライン
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-ja-en")

# 文字起こし結果の英語翻訳
english_transcription = translator(transcription_text, max_length=512)[0][
    "translation_text"
]
# with open("transcription_en.txt", "w", encoding="utf-8") as f:
#    f.write(english_transcription)

translation_en_path = os.path.join(output_dir, "translation_en.txt")
with open(translation_en_path, "w", encoding="utf-8") as f:
    f.write(english_transcription)

# 要約結果の英語翻訳
english_summary = translator(summary_text, max_length=512)[0]["translation_text"]
# with open("summary_en.txt", "w", encoding="utf-8") as f:
#    f.write(english_summary)

summary_en_path = os.path.join(output_dir, "summary_en.txt")
with open(summary_en_path, "w", encoding="utf-8") as f:
    f.write(english_summary)

print("✅ 英語翻訳が完了しました。")

# -------------------------
# 4. o3用プロンプト付き文書の作成
# -------------------------
print("📑 o3用のプロンプト付き文書を作成中...")

o3_prompt = (
    "以下は論文のアイデアの音声ファイルから文字起こしした内容です。\n"
    "内容について、o3の視点から意見や改善点を教えてください。\n\n"
)
o3_document = o3_prompt + transcription_text

# o3用文書を保存
o3_prompt_path = os.path.join(output_dir, "o3_prompt_document.txt")
with open(o3_prompt_path, "w", encoding="utf-8") as f:
    f.write(o3_document)

print("✅ o3用の文書作成が完了しました。")

# -------------------------
# 完了メッセージ
# -------------------------
print("🎉 すべての処理が完了しました！")

📢 文字起こしを開始します...


音声処理中...:   0%|          | 0/3 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


✅ 文字起こしが完了しました。
📄 要約を作成中...


要約処理中...:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use mps:0


✅ 要約が完了しました。
🌍 英語翻訳を開始します...


翻訳処理中...:   0%|          | 0/3 [00:00<?, ?it/s]

/Users/hajime/Library/Caches/pypoetry/virtualenvs/whisper-meeting-TBf8Z80e-py3.12/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use mps:0
Your input_length: 510 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


✅ 英語翻訳が完了しました。
📑 o3用のプロンプト付き文書を作成中...
✅ o3用の文書作成が完了しました。
🎉 すべての処理が完了しました！
